In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string



In [2]:
datapath = "/Users/aavattikutis/Documents/epidemicmodel/cccruns/fits/run2/tables/"
models = ["fulllinearmodel_fit_table.csv","reducedlinearmodelNegBinom_fit_table.csv",
          "reducedlinearmodelq0_fit_table.csv","reducedlinearmodelq0ctime_fit_table.csv",
         "nonlinearmodelq0ctime_fit_table.csv"]
model_hash = {}
k = -1
for model in models:
    k += 1
    model_hash[model] = string.ascii_uppercase[k]

df = pd.DataFrame.from_dict(model_hash, orient='index')
df.to_csv('../postmodel_derivatives/model_hash.csv', header=False)

In [3]:
rois = []
for model in models:
    df = pd.read_csv(datapath + model) #get rois in all tables (some may have failed)
    rois += list(df.roi.unique())

    
rois = list(set(rois))

#get inferred
theta = df.columns[2:] 
ntheta = len(theta)



In [4]:
#plot side by side 

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
             
roi_us = np.sort([i for i in rois if i[:2]=='US'])#[::-1]
roi_other = np.sort([i for i in rois if i[:2]!='US'])#[::-1]
rois = list(roi_us) + list(roi_other)

dfwaic = pd.DataFrame(columns=['roi', 'model', 'waic'])

k = 0
for roi in rois:
#     print(roi)
    for model in models:
        k += 1
        df = pd.read_csv(datapath + model)
        try:
            measure = df.loc[(df.roi==roi)&(df['quantile']==0.5),'waic'].values[0]
#             dfwaic.loc[k] = [roi,model_hash[model],measure]
            dfwaic.loc[k] = [roi,model.split('.csv')[0],measure]
        except:
#             dfwaic.loc[k] = [roi,model_hash[model],1e6]
            dfwaic.loc[k] = [roi,model.split('.csv')[0],1e6]



US
US_AK
US_AR
US_AZ
US_DC
US_DE
US_GU
US_HI
US_IA
US_KY
US_MD
US_ME
US_MI
US_MN
US_MT
US_ND
US_NH
US_NJ
US_NM
US_NY
US_OK
US_RI
US_SD
US_TN
US_TX
US_UT
US_VA
US_VI
US_VT
US_WV
US_WY
Afghanistan
Albania
Algeria
Andorra
Antigua and Barbuda
Argentina
Armenia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Benin
Bolivia
Bosnia and Herzegovina
Brazil
Brunei
Bulgaria
Burkina Faso
Burma
Cabo Verde
Cameroon
Chile
China
Colombia
Congo (Brazzaville)
Congo (Kinshasa)
Costa Rica
Croatia
Cuba
Cyprus
Czechia
Denmark
Diamond Princess
Djibouti
Dominican Republic
Ecuador
Egypt
El Salvador
Estonia
Ethiopia
Finland
France
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Guatemala
Guinea
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Korea, South
Kosovo
Kuwait
Kyrgyzstan
Latvia
Lebanon
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
Malaysia
Mali
Malta
Mauritius
Mexico
Moldova
Monaco
Montenegro
Morocco
Netherland

In [5]:
dfwaic_sorted = dfwaic.sort_values(['waic'],ascending=True).groupby('roi')
# dfwaic_sorted = dfwaic_sorted.head().reset_index(drop=True)
productname = "roimodel_waicsorted_topmodel_modelname.csv"
dfwaic_sorted.head(1).to_csv('../postmodel_derivatives/'+productname,index=False)